In [2]:
%pip install langchain-core langchain-community

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [1]:
MODEL = 'llama2'


In [4]:

from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings=OllamaEmbeddings()


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [31]:
!ls

01-run-ollama.ipynb  02-langchain.ipynb


In [5]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("../pdfs/gpus.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='Comparison of the Best GPUs on the Market\nNVIDIA GeForce RTX 4090\nSpecifications:\n- CUDA Cores: 16,384\n- Base Clock: 2.23 GHz\n- Boost Clock: 2.52 GHz\n- Memory: 24 GB GDDR6X\n- Memory Bandwidth: 1,008 GB/s\n- TDP: 450W\n- Ray Tracing Cores: 144\n- Tensor Cores: 576\nAdvantages:\n- Unmatched Performance: The RTX 4090 offers the highest performance in gaming and professional applications,\ncapable of 4K gaming at high frame rates.\n- Ray Tracing and AI: Exceptional ray tracing capabilities and AI-based features like DLSS 3.0 enhance both visual\nquality and performance.\n- Future-Proof: With a large amount of VRAM and advanced features, the RTX 4090 is highly future-proof for upcoming\ngames and applications.\nDisadvantages:\n- High Power Consumption: The 450W TDP requires a robust power supply and efficient cooling solutions.\n- Expensive: One of the most expensive GPUs on the market, making it less accessible for budget-conscious users.\n- Size: Large form 

In [6]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
# print(prompt.format(context="Here is some context", question="This is the question"))



In [7]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()



{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [8]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)

In [12]:
retriever = vectorstore.as_retriever()
retriever.invoke("ML")

[Document(page_content='Comparison of the Best GPUs on the Market\nNVIDIA GeForce RTX 4090\nSpecifications:\n- CUDA Cores: 16,384\n- Base Clock: 2.23 GHz\n- Boost Clock: 2.52 GHz\n- Memory: 24 GB GDDR6X\n- Memory Bandwidth: 1,008 GB/s\n- TDP: 450W\n- Ray Tracing Cores: 144\n- Tensor Cores: 576\nAdvantages:\n- Unmatched Performance: The RTX 4090 offers the highest performance in gaming and professional applications,\ncapable of 4K gaming at high frame rates.\n- Ray Tracing and AI: Exceptional ray tracing capabilities and AI-based features like DLSS 3.0 enhance both visual\nquality and performance.\n- Future-Proof: With a large amount of VRAM and advanced features, the RTX 4090 is highly future-proof for upcoming\ngames and applications.\nDisadvantages:\n- High Power Consumption: The 450W TDP requires a robust power supply and efficient cooling solutions.\n- Expensive: One of the most expensive GPUs on the market, making it less accessible for budget-conscious users.\n- Size: Large form 

In [42]:
output = chain.invoke({"context":"The name I was given was Deepak", "question":"What's my name"})
print(output)
# for token in output:
#     print(token, end="", flush=True)

 Based on the context provided, your name is Deepak.


In [13]:

from operator import itemgetter



chain = (
    {
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question")
    } | prompt | model | parser)
chain.input_schema.schema()


{'title': 'RunnableParallel<context,question>Input',
 'type': 'object',
 'properties': {'question': {'title': 'Question'}}}

In [14]:
questions = [
    "What is this document about?",
    "What GPUs are being compared?",
]


In [26]:
for question in questions:
    print(f"Question: {question}")
    print(f"Answer: ", end="")
    for token in chain.stream({"question" : question}):
        print(token, end="", flush=True)
        
    print()
    

Question: What is this document about?
Answer: Based on the content provided in the document, it appears to be a comparison of various high-end graphics processing units (GPUs) available in the market, specifically focusing on their performance, features, and specifications. The document provides detailed information on each of the GPUs discussed, including their CUDA cores, base and boost clock speeds, memory capacity, memory bandwidth, TDP, ray tracing cores, tensor cores, and other features.

The document starts by discussing the advantages and disadvantages of each GPU, followed by a detailed comparison of their specifications and performance in different areas. The document concludes by summarizing the key points and providing recommendations for users based on their specific needs and budget constraints.
Question: What GPUs are being compared?
Answer: Based on the context provided, the following GPUs are being compared:

1. NVIDIA GeForce RTX 4090
2. AMD Radeon RX 7900 XTX
3. NVI